In [ ]:
#| default_exp main
import sys,os
from fastcore.xtras import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

# Principal
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [ ]:
#| export
import json
from datetime import datetime
from pathlib import Path
from typing import Union

import pandas as pd
import pyodbc
from dotenv import find_dotenv, load_dotenv
from fastcore.test import *
from pymongo import MongoClient
from rich import print

from extracao.format import _filter_matlab, _format_matlab, merge_on_frequency

load_dotenv(find_dotenv(), override=True)

True

In [ ]:
#|eval: false
folder = Path.cwd().parent / 'extracao' / 'datasources' / 'arquivos' / 'saida'
folder.ls()

(#21) [Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/aero.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/aero_discarded.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/AnatelDB.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/arquitetura.md'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/base.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/canalização_smp.xlsx'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/dados.md'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/estacoes.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/estacoes_disca

In [ ]:
#|eval: false
#|code_fold: true

def read_df(name: str, dtype_backend='pyarrow')->pd.DataFrame:
    return pd.read_parquet(folder / f'{name}.parquet.gzip', dtype_backend=dtype_backend)# .astype("category")

In [ ]:
estacoes = read_df('estacoes', 'numpy_nullable')
estacoes

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,0.028,FURNAS CENTRAIS ELETRICAS S A,01030052263,019,1557670,-22.662778,-43.476389,3303500,Nova Iguaçu,RJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,019,1558412,-23.71,-46.273333,3530607,Mogi das Cruzes,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,019,1557823,-23.441667,-46.590833,3550308,São Paulo,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,019,859761,-22.926667,-43.265,3304557,Rio de Janeiro,RJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.03,FURNAS CENTRAIS ELETRICAS S A,01030052263,019,859753,-22.774167,-47.004444,3509502,Campinas,SP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979517,84781.25,TIM S A,50417425295,019,1015225680,-19.7375,-47.927639,3170107,Uberaba,MG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
979518,84781.25,TIM S A,50417425295,019,1015020205,-6.064444,-49.907528,1505536,Parauapebas,PA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
979519,84781.25,TIM S A,50417425295,019,1015225729,-2.478361,-44.17475,2107506,Paço do Lumiar,MA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
979520,84812.5,COMPUTADORES E SISTEMAS LTDA ...,50433753536,019,1011928598,-12.9816888888889,-38.4543583333333,2927408,Salvador,BA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
estacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979522 entries, 0 to 979521
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   Frequência                   979522 non-null  category
 1   Entidade                     979521 non-null  category
 2   Fistel                       978101 non-null  category
 3   Serviço                      978101 non-null  category
 4   Estação                      978101 non-null  category
 5   Latitude                     979522 non-null  category
 6   Longitude                    979522 non-null  category
 7   Código_Município             979522 non-null  category
 8   Município                    979522 non-null  category
 9   UF                           979522 non-null  category
 10  Classe                       977680 non-null  category
 11  Classe_Emissão               977680 non-null  category
 12  Largura_Emissão(kHz)         977680 non-null

In [ ]:
estacoes.Fonte.value_counts()

Fonte
MOSAICO-LICENCIAMENTO                              817750
CANALIZACAO-SMP                                    119408
STEL                                                19599
MOSAICO-SRD                                         14855
SRD                                                  4965
STEL | ICAO                                           904
ICAO                                                  690
CANALIZACAO-VOR/ILS/DME                               264
MOSAICO-LICENCIAMENTO | ICAO | AISWEB                 173
AISGEO                                                169
RADAR                                                 152
MOSAICO-LICENCIAMENTO | ICAO                          152
STEL | ICAO | AISWEB                                  142
AISWEB                                                 58
MOSAICO-LICENCIAMENTO | AISGEO                         52
STEL | AISWEB                                          35
ICAO | AISWEB                                          32
ICAO | A

In [ ]:
estacoes.loc[estacoes.Estação == '322609933', 'Log'].item()

'[("Colunas", "Designação_Emissão"]),          ("Processamento", "Registro expandido nos componentes individuais e extraídas Largura e Classe")]|[("Colunas", ["Código_Município", "Município", "UF"]),   ("Processamento", "Informações substituídas  pela localização correta das coordenadas.")        '

In [ ]:
estacoes['Frequência'] = estacoes.Frequência.astype('category')
estacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979522 entries, 0 to 979521
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype                                                       
---  ------                       --------------   -----                                                       
 0   Frequência                   979522 non-null  category                                                    
 1   Entidade                     979522 non-null  string[pyarrow]                                             
 2   Fistel                       979522 non-null  uint64[pyarrow]                                             
 3   Serviço                      979522 non-null  uint16[pyarrow]                                             
 4   Estação                      979522 non-null  string[pyarrow]                                             
 5   Latitude                     979522 non-null  float[pyarrow]                                        

In [ ]:
cat = estacoes.astype('category').copy()
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979522 entries, 0 to 979521
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   Frequência                   979522 non-null  category
 1   Entidade                     979522 non-null  category
 2   Fistel                       979522 non-null  category
 3   Serviço                      979522 non-null  category
 4   Estação                      979522 non-null  category
 5   Latitude                     979522 non-null  category
 6   Longitude                    979522 non-null  category
 7   Código_Município             979522 non-null  category
 8   Município                    979522 non-null  category
 9   UF                           979522 non-null  category
 10  Classe                       979522 non-null  category
 11  Classe_Emissão               979522 non-null  category
 12  Largura_Emissão(kHz)         979522 non-null

In [ ]:
cat.to_parquet(folder / 'estacoes_cat.parquet.gzip', index=False, compression='gzip')

In [ ]:
estacoes.Fonte.value_counts()

Fonte
MOSAICO-LICENCIAMENTO                              817750
CANALIZACAO-SMP                                    119408
STEL                                                19599
MOSAICO-SRD                                         14855
SRD                                                  4965
STEL | ICAO                                           904
ICAO                                                  690
CANALIZACAO-VOR/ILS/DME                               264
MOSAICO-LICENCIAMENTO | ICAO | AISWEB                 173
AISGEO                                                169
MOSAICO-LICENCIAMENTO | ICAO                          152
RADAR                                                 152
STEL | ICAO | AISWEB                                  142
AISWEB                                                 58
MOSAICO-LICENCIAMENTO | AISGEO                         52
STEL | AISWEB                                          35
ICAO | AISWEB                                          32
ICAO | A

In [ ]:
for c in [
	'ICAO',
	'CANALIZACAO-VOR/ILS/DME',
	'AISGEO',
	'RADAR',
	'ICAO | AISWEB',
	'AISWEB',
	'ICAO | AISGEO',
	'REDEMET',
]:
	for col in [
		'Fistel',
		'Serviço',
		'Estação',
		'Classe',
		'Classe_Emissão',
		'Largura_Emissão(kHz)',
		'Validade_RF',
		'Status',
		'Fonte',
		'Multiplicidade',
		'Cota_Base_Torre(m)',
		'Potência_Transmissor(W)',
		'Ganho_Antena(dBd)',
		'Ângulo_Elevação_Antena',
		'Azimute_Antena',
		'Altura_Antena(m)',
		'Atenuação_Linha(db/100m)',
		'Perdas_Acessórias_Linha(db)',
		'Padrão_Antena(dBd)',
		'Comprimento_Linha(m)',
		'Relatório_Canal',
	]:
		assert estacoes.loc[estacoes.Fonte == c, col].nunique() == 1


In [ ]:
for c in [
	'STEL | ICAO',
	'MOSAICO-LICENCIAMENTO | ICAO | AISWEB',
	'MOSAICO-LICENCIAMENTO | ICAO',
	'STEL | ICAO | AISWEB',
	'STEL | AISWEB',
	'MOSAICO-LICENCIAMENTO | AISWEB',
]:
	print(estacoes.loc[estacoes.Fonte == c, ['Fistel', 'Serviço', 'Estação']].sample(5))


Fistel  Serviço     Estação
37302  50415208017      507  1011658779
37214  50405707169      507  1009310221
36845  50004641507      507  1006018902
36467  50415723132      507  1007719025
38021  50415723132      507  1007724282

Fistel  Serviço    Estação
35502  11030016470       19  699526159
35556  11030016470       19  699526302
35867  11030016470       19  699525314
35614  50013784196       19  691104018
35541  11020237015       19    1068946

Fistel  Serviço     Estação
37611  50413129829       19  1002318553
37022   2030398152       19   687305675
35644  50406881138       19   692616217
37077  50404959350       19   690327978
37106  50411781430       19  1000665175

Fistel  Serviço     Estação
35878  50415723132      507  1007685023
36107  50415723132      507  1007707558
37902  50415723132      507  1007541412
35602  50415723132      507  1007688634
37930  50415723132      507  1007661485

Fistel  Serviço     Estação
35933  50415723132      507  1007558129
36102  50415723132      507  1007640518
35621  50415723132      507  1007541412
36122  50415723132      507  1007688634
36851  50415699916      507  1005348232

Fistel  Serviço     Estação
36427  11030016470       19   699525039
27001  50444239804      108  1000823587
35694  11030016470       19   699525250
27009  50444239804      108  1000823501
27008  50444239804      108  1000823439

In [ ]:
df

,Frequência,Entidade_x,Fistel,Serviço,Estação,Latitude_x,Longitude_x,Código_Município,Município,UF,...,Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal,Latitude_y,Longitude_y,Entidade_y,Fonte_y,Multiplicidade_y,_merge
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066219,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-23.631379722,-46.652060278,DME - AEROPORTO DE CONGONHAS 34X,AISGEO,1,right_only
1066220,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-5.771808333,-35.350191667,DME - SAO GONCALO 34X,AISGEO,1,right_only
1066221,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-9.336946111,-54.952427778,DME - CACHIMBO 34X,AISGEO,1,right_only
1066222,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-9.509804722,-35.786944417,DME - IMC 34X,AISGEO,1,right_only


In [ ]:
left_suffix, right_suffix = '_x', '_y'
lat, long = 'Latitude', 'Longitude'

left_only = df._merge == 'left_only'
right_only = df._merge == 'right_only'
both = df._merge == 'both'
df = df.drop(columns=['_merge'])

In [ ]:
only_left = df[left_only].copy()
only_left = only_left.iloc[:, :len(df_left.columns)]
only_left.columns = df_left.columns
only_left


,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065599,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578503,-6.31095,-38.2823,2406007,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065600,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578511,-6.40588,-38.3899,2407005,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065601,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578520,-6.45823,-38.182,2414100,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065602,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578538,-5.71287,-38.1578,2311231,<NA>,CE,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
only_right = df[right_only].copy()
cols = ['Frequência'] + df.columns[len(df_left.columns):].to_list()
only_right = only_right.loc[:, cols]
only_right.columns = df_right.columns
only_right

,Frequência,Latitude,Longitude,Entidade,Fonte,Multiplicidade
1065604,0.1143,-32.340055556,-54.223888889,NDB - MELO,AISGEO,1
1065605,0.285,-21.267052778,-43.764111111,NDB - BARBACENA NDB NOT AVBL BEYOND 60 NM,AISGEO,1
1065606,0.285,-22.197333333,-54.928333333,NDB - DOURA NDB NOT AVBL BEYOND 70 NM,AISGEO,1
1065607,0.375,-19.016219444,-57.664455556,NDB - CORUMBA,AISGEO,1
1065608,0.385,-16.441666667,-39.0825,NDB - PORTO SEGURO NDB NOT AVBL BEYOND 27 NM,AISGEO,1
...,...,...,...,...,...,...
1066219,995.0,-23.631379722,-46.652060278,DME - AEROPORTO DE CONGONHAS 34X,AISGEO,1
1066220,995.0,-5.771808333,-35.350191667,DME - SAO GONCALO 34X,AISGEO,1
1066221,995.0,-9.336946111,-54.952427778,DME - CACHIMBO 34X,AISGEO,1
1066222,995.0,-9.509804722,-35.786944417,DME - IMC 34X,AISGEO,1
